In [2]:
import os
import sys
import pickle
import csv
import copy

import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [4]:
sys.path.append("../../../tfobjdetect/lib")
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

../../../tfobjdetect/lib/object_detection/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch

## Loading and preparing label maps

In [5]:
with open('../../temp/label_matching_coco_oi_ws_300.pkl', 'rb') as fin:
    labelmap = pickle.load(fin)

In [6]:
idx2mid_coco = labelmap['idx2mid_coco']
idx2mid_oi = labelmap['idx2mid_oi']
idx2mid_ws = labelmap['idx2mid_ws']
mid2idx_coco = labelmap['mid2idx_coco']
mid2idx_oi = labelmap['mid2idx_oi']
mid2idx_ws = labelmap['mid2idx_ws']
mid2name_coco = labelmap['mid2name_coco']
mid2name_oi = labelmap['mid2name_oi']
mid2name_ws = labelmap['mid2name_ws']
name2mid_coco = labelmap['name2mid_coco']
name2mid_oi = labelmap['name2mid_oi']
name2mid_ws = labelmap['name2mid_ws']
common_mids_coco = labelmap['common_mids_coco']
common_mids_ws = labelmap['common_mids_ws']


In [7]:
coco_mids = set(mid2idx_coco.keys())
oi_mids = set(mid2idx_oi.keys())
ws_mids = set(mid2idx_ws.keys())


In [8]:
mid2name_all = {}
for key in coco_mids:
    mid2name_all[key] = mid2name_coco[key]
for key in oi_mids:
    mid2name_all[key] = mid2name_oi[key]
for key in ws_mids:
    mid2name_all[key] = mid2name_ws[key]

In [9]:
with open('../../temp/all_labels_coco_oi_ws_300.pkl', 'rb') as fin:
    all_labels = pickle.load(fin)

In [10]:
with open('../../results/det_results_postproc_11.pkl', 'rb') as fin:
    det_results_sorted = pickle.load(fin)

## Choosing examples for each class to visualize

In [11]:
label_to_img = {key: [] for key in all_labels}
for key, val in det_results_sorted.items():
    for ii in range(val['num_detections']):
        if val['detection_scores'][ii] < 0.35:
            continue
        label_to_img[val['detection_classes'][ii]].append(key)

In [12]:
idx = np.argsort([-len(label_to_img[label]) for label in all_labels])

In [13]:
for i in idx:
    print(all_labels[i], mid2name_all[all_labels[i]], len(label_to_img[all_labels[i]]))

/m/01g317 Person 2089
/m/04yx4 Man 904
/m/0dzct Human face 588
/m/07r04 Truck 298
/m/03bt1vf Woman 287
/m/05s2s Plant 269
/m/0k4j Car 260
/m/01prls Land vehicle 161
/m/0c_jw Furniture 147
/m/0hf58v5 Luggage and bags 135
/m/07cmd Tank 127
/m/01n5jq Poster 91
/m/012t_z Businessperson 79
/m/03120 Flag 78
/m/0cgh4 Building 72
/m/0jbk Animal 64
/m/078my Trailer truck 57
/m/01xr66 Spokesperson 57
/m/07jdr Train 53
/m/02wbm Food 49
/m/07s6nbt Text 46
/m/07k1x Tool 45
/m/0jyfg Glasses 44
/m/07yv9 Vehicle 43
/m/050k8 Mobile phone 39
/m/0dwx7 Logo 38
/m/04_tb Map 36
/m/0zvk5 Helmet 33
/m/019jd Boat 33
/m/083kb Weapon 32
/m/0gvss07 Combat vehicle 32
/m/035y33 Official 31
/m/03jm5 House 31
/m/0728h Self-propelled artillery 31
/m/06c54 Rifle 31
/m/0bt9lr Dog 30
/m/01fnck Waste 30
/m/0fkwjg Commercial vehicle 27
/m/03qtwd Crowd 26
/m/01xgg_ Medical equipment 26
/m/0cmf2 Airplane 25
/m/070qm Freight transport 24
/m/0g54v5d Military vehicle 24
/m/01knjb Billboard 24
/m/0h2r6 Van 24
/m/06q40 Smoke 23
/

In [14]:
select_labels = ['/m/01fnck', '/m/019jd', '/m/03120', '/m/02_41', '/m/07cmd', '/m/06q40', '/m/03qtwd', '/m/09x0r', '/m/01xgg_', '/m/08qrwn', '/m/012n4x', '/m/02p16m6', '/m/02lbcq', '/m/09ct_', '/m/0g54v5d', '/m/0bg2p', '/m/0ct4f', '/m/01nd_n', '/m/09rvcxw', '/m/01nl4x', '/m/04ctx', '/m/04ylt', '/m/0gvss07', '/m/01rzcn', '/m/06nrc', '/m/0cyfs', '/m/0f5lx', '/m/04zjc', '/m/0dhz0', '/m/01lcw4', '/m/03htg', '/m/0gxl3', '/m/01bq8v', '/m/0lt4_', '/m/02gzp', '/m/02yjc', '/m/012n7d', '/m/0jb3', '/m/0dwx7']

## Visualizing Results

In [15]:
mid2name_extended = {}
extended_classes = set()
for key in coco_mids:
    for model, abv in [('coco', 'CO'), ('oi', 'OI'), ('ws', 'WS'), ('coco/J', 'CO/J'), ('oi/J', 'OI/J'), ('ws/J', 'WS/J')]:
        extended_classes.add(key + '/' + model)
        mid2name_extended[key + '/' + model] = mid2name_coco[key] + f' ({abv})'
for key in oi_mids:
    for model, abv in [('coco', 'CO'), ('oi', 'OI'), ('ws', 'WS'), ('coco/J', 'CO/J'), ('oi/J', 'OI/J'), ('ws/J', 'WS/J')]:
        extended_classes.add(key + '/' + model)
        mid2name_extended[key + '/' + model] = mid2name_oi[key] + f' ({abv})'
for key in ws_mids:
    for model, abv in [('coco', 'CO'), ('oi', 'OI'), ('ws', 'WS'), ('coco/J', 'CO/J'), ('oi/J', 'OI/J'), ('ws/J', 'WS/J')]:
        extended_classes.add(key + '/' + model)
        mid2name_extended[key + '/' + model] = mid2name_ws[key] + f' ({abv})'
extended_classes = list(extended_classes)
mid2idx_extended = {}
for i, key in enumerate(extended_classes):
    mid2idx_extended[key] = i
category_index_extended = {mid2idx_extended[key]:{'id': mid2idx_extended[key], 'name': mid2name_extended[key]} for key in mid2name_extended}

In [16]:
det_results_forviz = copy.deepcopy(det_results_sorted)

In [17]:
for key in det_results_forviz:
    for i in range(len(det_results_forviz[key]['detection_classes'])):
        label = det_results_forviz[key]['detection_classes'][i] + '/' + det_results_forviz[key]['detection_models'][i]
        det_results_forviz[key]['detection_classes'][i] = mid2idx_extended[label]


In [18]:
print(det_results_forviz[key]['detection_classes'])

[3993, 2320, 430, 2571, 3147, 342, 3615, 73, 73, 73, 73, 676, 199, 1266, 882, 186, 4337, 1338, 3883, 1394, 1238, 4133, 3907, 946, 1757, 2894, 2807, 2976, 944, 2757, 1730, 1343, 1007, 3505, 2958, 925, 1007, 3034, 2624, 1323, 3034, 2195, 2624, 3828, 1882, 1995, 422, 3466, 1882, 3130, 3534, 2926, 3352, 1948, 1864, 4075, 1101, 251, 1799, 1865, 622, 3352, 4181, 2339, 667, 1544, 673, 861, 2695, 1899, 1101, 295, 2485, 2132, 2401, 3246, 3361, 3705, 2132, 558, 424, 3660, 3246, 193, 73, 3194, 2583, 1747, 2125, 832, 2692, 4314, 818, 73, 73, 73, 73, 73, 3993, 73]


In [19]:
def preproc(im):
    target_size = 400
    max_size = 1024
    im_size_min = np.min(im.shape[0:2])
    im_size_max = np.max(im.shape[0:2])
    im_scale = float(target_size) / float(im_size_min)
    # Prevent the biggest axis from being more than MAX_SIZE
    if np.round(im_scale * im_size_max) > max_size:
        im_scale = float(max_size) / float(im_size_max)
    im = cv2.resize(im, None, None, fx=im_scale, fy=im_scale,
                    interpolation=cv2.INTER_LINEAR)    
    return im

In [20]:
def show(imgid, thresh):
    filename = '../../../../data/seedling/seedling_part2/jpg/' + imgid
    print(imgid)
    with open(filename, 'rb') as fin:
        _ = fin.read(1024)
        imgbin = fin.read()
    imgbgr = cv2.imdecode(np.fromstring(imgbin, dtype='uint8'), cv2.IMREAD_COLOR)
    image_np = imgbgr[:,:,[2,1,0]]
    image_np = preproc(image_np)
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      np.asarray(det_results_forviz[imgid]['detection_boxes_normalized']),
      det_results_forviz[imgid]['detection_classes'],
      det_results_forviz[imgid]['detection_scores'],
      category_index_extended,
      instance_masks=det_results_forviz[imgid].get('detection_masks'),
      use_normalized_coordinates=True,
      min_score_thresh=thresh,
      line_thickness=2)
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np)
    plt.axis('off')
    plt.show()    

In [21]:
for label in select_labels:
    if len(label_to_img[label]) == 0:
        continue
    print(mid2name_all[label])
    imgid = np.random.choice(label_to_img[label])
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.35, continuous_update=False), 
        imgid=fixed(imgid)
    )
    w.children[-1].layout.height = '600px'
    display(w)

Waste


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Boat


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Flag


...

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Assault rifle


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Logo


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

In [22]:
for label in select_labels:
    if len(label_to_img[label]) == 0:
        continue
    print(mid2name_all[label])
    imgid = np.random.choice(label_to_img[label])
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.35, continuous_update=False), 
        imgid=fixed(imgid)
    )
    w.children[-1].layout.height = '600px'
    display(w)

Waste


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Boat


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Flag


...

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Assault rifle


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Logo


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

In [23]:
for label in select_labels:
    if len(label_to_img[label]) == 0:
        continue
    print(mid2name_all[label])
    imgid = np.random.choice(label_to_img[label])
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.35, continuous_update=False), 
        imgid=fixed(imgid)
    )
    w.children[-1].layout.height = '600px'
    display(w)

Waste


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Boat


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Flag


...

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Assault rifle


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Logo


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

In [24]:
for label in select_labels:
    if len(label_to_img[label]) == 0:
        continue
    print(mid2name_all[label])
    imgid = np.random.choice(label_to_img[label])
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.35, continuous_update=False), 
        imgid=fixed(imgid)
    )
    w.children[-1].layout.height = '600px'
    display(w)

Waste


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Boat


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Flag


...

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Assault rifle


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Logo


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…